## LLM Ops Stack  - LiteLLM Proxy + Langfuse 

This notebook demonstrates how to use LiteLLM Proxy with Langfuse 
- Use LiteLLM Proxy for calling 100+ LLMs in OpenAI format
- Use Langfuse for viewing request / response traces 


In this notebook we will setup LiteLLM Proxy to make requests to OpenAI, Anthropic, Bedrock and automatically log traces to Langfuse.

## 1. Setup LiteLLM Proxy

### 1.1 Define .env variables 
Define .env variables on the container that litellm proxy is running on.
```bash
## LLM API Keys
OPENAI_API_KEY=sk-proj-1234567890
ANTHROPIC_API_KEY=sk-ant-api03-1234567890
AWS_ACCESS_KEY_ID=1234567890
AWS_SECRET_ACCESS_KEY=1234567890

## Langfuse Logging 
LANGFUSE_PUBLIC_KEY="pk-lf-xxxx9"
LANGFUSE_SECRET_KEY="sk-lf-xxxx9"
LANGFUSE_HOST="https://us.cloud.langfuse.com"
```


### 1.1 Setup LiteLLM Proxy Config yaml 
```yaml
model_list:
  - model_name: gpt-4o
    litellm_params:
      model: openai/gpt-4o
      api_key: os.environ/OPENAI_API_KEY
  - model_name: claude-3-5-sonnet-20241022
    litellm_params:
      model: anthropic/claude-3-5-sonnet-20241022
      api_key: os.environ/ANTHROPIC_API_KEY
  - model_name: us.amazon.nova-micro-v1:0
    litellm_params:
      model: bedrock/us.amazon.nova-micro-v1:0
      aws_access_key_id: os.environ/AWS_ACCESS_KEY_ID
      aws_secret_access_key: os.environ/AWS_SECRET_ACCESS_KEY

litellm_settings:
  callbacks: ["langfuse"]


```

## 2. Make LLM Requests to LiteLLM Proxy

Now we will make our first LLM request to LiteLLM Proxy

### 2.1 Setup Client Side Variables to point to LiteLLM Proxy
Set `LITELLM_PROXY_BASE_URL` to the base url of the LiteLLM Proxy and `LITELLM_VIRTUAL_KEY` to the virtual key you want to use for Authentication to LiteLLM Proxy. (Note: In this initial setup you can)

In [23]:

LITELLM_PROXY_BASE_URL="http://0.0.0.0:4000"
LITELLM_VIRTUAL_KEY="sk-oXXRa1xxxxxxxxxxx"

In [22]:
import openai
client = openai.OpenAI(
    api_key=LITELLM_VIRTUAL_KEY,
    base_url=LITELLM_PROXY_BASE_URL
)

response = client.chat.completions.create(
    model="gpt-4o",
    messages = [
        {
            "role": "user",
            "content": "what is Langfuse?"
        }
    ],
)

response

ChatCompletion(id='chatcmpl-B0sq6QkOKNMJ0dwP3x7OoMqk1jZcI', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Langfuse is a platform designed to monitor, observe, and troubleshoot AI and large language model (LLM) applications. It provides features that help developers gain insights into how their AI systems are performing, make debugging easier, and optimize the deployment of models. Langfuse allows for tracking of model interactions, collecting telemetry, and visualizing data, which is crucial for understanding the behavior of AI models in production environments. This kind of tool is particularly useful for developers working with language models who need to ensure reliability and efficiency in their applications.', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))], created=1739550502, model='gpt-4o-2024-08-06', object='chat.completion', service_tier='default', system_fingerprint='fp_523b9b6e5f', usa

### 2.3 View Traces on Langfuse
LiteLLM will send the request / response, model, tokens (input + output), cost to Langfuse.

![image_description](litellm_proxy_langfuse.png)

### 2.4 Call Anthropic, Bedrock models 

Now we can call `us.amazon.nova-micro-v1:0` and `claude-3-5-sonnet-20241022` models defined on your config.yaml both in the OpenAI request / response format.

In [24]:
import openai
client = openai.OpenAI(
    api_key=LITELLM_VIRTUAL_KEY,
    base_url=LITELLM_PROXY_BASE_URL
)

response = client.chat.completions.create(
    model="us.amazon.nova-micro-v1:0",
    messages = [
        {
            "role": "user",
            "content": "what is Langfuse?"
        }
    ],
)

response

ChatCompletion(id='chatcmpl-7756e509-e61f-4f5e-b5ae-b7a41013522a', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="Langfuse is an observability tool designed specifically for machine learning models and applications built with natural language processing (NLP) and large language models (LLMs). It focuses on providing detailed insights into how these models perform in real-world scenarios. Here are some key features and purposes of Langfuse:\n\n1. **Real-time Monitoring**: Langfuse allows developers to monitor the performance of their NLP and LLM applications in real time. This includes tracking the inputs and outputs of the models, as well as any errors or issues that arise during operation.\n\n2. **Error Tracking**: It helps in identifying and tracking errors in the models' outputs. By analyzing incorrect or unexpected responses, developers can pinpoint where and why errors occur, facilitating more effective debugging and improvemen

## 3. Advanced - Set Langfuse Trace ID, Tags, Metadata 

Here is an example of how you can set Langfuse specific params on your client side request. See full list of supported langfuse params [here](https://docs.litellm.ai/docs/observability/langfuse_integration)

You can view the logged trace of this request [here](https://us.cloud.langfuse.com/project/clvlhdfat0007vwb74m9lvfvi/traces/567890?timestamp=2025-02-14T17%3A30%3A26.709Z)

In [27]:
import openai
client = openai.OpenAI(
    api_key=LITELLM_VIRTUAL_KEY,
    base_url=LITELLM_PROXY_BASE_URL
)

response = client.chat.completions.create(
    model="us.amazon.nova-micro-v1:0",
    messages = [
        {
            "role": "user",
            "content": "what is Langfuse?"
        }
    ],
    extra_body={
        "metadata": {
            "generation_id": "1234567890",
            "trace_id": "567890",
            "trace_user_id": "user_1234567890",
            "tags": ["tag1", "tag2"]
        }
    }
)

response

ChatCompletion(id='chatcmpl-789babd5-c064-4939-9093-46e4cd2e208a', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="Langfuse is an observability platform designed specifically for monitoring and improving the performance of natural language processing (NLP) models and applications. It provides developers with tools to track, analyze, and optimize how their language models interact with users and handle natural language inputs.\n\nHere are some key features and benefits of Langfuse:\n\n1. **Real-Time Monitoring**: Langfuse allows developers to monitor their NLP applications in real time. This includes tracking user interactions, model responses, and overall performance metrics.\n\n2. **Error Tracking**: It helps in identifying and tracking errors in the model's responses. This can include incorrect, irrelevant, or unsafe outputs.\n\n3. **User Feedback Integration**: Langfuse enables the collection of user feedback directly within the p

## 